# GIS2RHESSys



**Introduction:**

GIS2RHESSys is designed to setup RHESSysEastCoast model. There are two parts in this model setup process. First part is to setup GIS dataset from the raw data user provide. GIS processes in the first part may include create GRASS dataset, import/reproject/resample elevation data, delineate catchment, analyze drainage & flow accumulation, set climate zones, identify riparian boundary, classify soil textures, import and translate LULC data to RHESSys model, distribute forest community, and identify surface preferential flow path and transports. The guide in the first part is in a list format, listing examples of modules for certain GIS processes. This is not a JupyterNotebook for users to execute codes listed below. Users may copy/paste the codes to assemble a workflow bash/shell script running on local machine.

The second part of model setup process, assumed GIS is well setup, is a template listing library files/parameter datasets and GIS maps/layers to build RHESSysEastCoast input files, include, def files, flow tables, worldfiles, and worldfile headers. Users need to provide climate data, obtain and complied the code of RHESSysEastCoast, and set "tecfile" to control the model output format. Model calibration is a seperated process (https://github.com/laurencelin/R-coded-scripts-for-RHESSys-calibration). 

**Software / computer requirements for GIS2RHESSys:**
- Max OS / LINUX (e.g., Urbantu) 
- GRASS 7.x installed and its associated libraries (see GRASS GIS website for details)
- R 3.6.x or above
- R package “rgrass7” and its dependences

## Headers and paths for all scripts

0. **Set system paths to GRASS, R, and Github libraries**

<div class="alert alert-block alert-info">
The PATH settings below are for MAC OS. Users may need to modify some paths for the LINUX environment.
</div>

In [ ]:
# path setup below is for Mac OS
# adding R paths
export PATH=$PATH:/usr/local/bin
export PATH=$PATH:/Library/Frameworks/R.framework/Resources
# adding GRASS bin paths
export PATH=$PATH:/Applications/GRASS-7.4.4.app/Contents/Resources/bin:/Applications/GRASS-7.4.4.app/Contents/Resources/scripts:/etc:/usr/lib
# adding GRASS environment variables: GISBASE_USER and GISBASE_SYSTEM in Grass.sh
export PATH=$PATH:~/Library/GRASS/7.4/Modules/bin:~/Library/GRASS/7.4/Modules/scripts
# adding GDAL and EPSG code lookup paths (Running gdal-config --datadir shows where GDAL searches for gcs.csv)
export GDAL_DATA=/Applications/GRASS-7.4.4.app/Contents/Resources/share/gdal
GITHUBLIBRARIES="https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/libraries"

<div class="alert alert-block alert-info">
Enter the path to the project directory.<br>
When search for UTM EPSG code, we recommend to use datum NAD83 for the U.S. catchments.<br>
The spatial resolution (e.g., 3/10/30 m) of the elevation raster is often the spatial resolution of the RHESSys model.
</div>

In [ ]:
PROJDIR='/Workspace/current_projects/Delight' # full path to the project location;
EPSGCODE='EPSG:26918' # EPSG code for NAD83 UTM ##N for the catchment
RESOLUTION=10 #spatial resolution (meters) of the grids
RHESSysNAME='rhessys' # e.g., rhessys
GISDBASE="$PROJDIR"/grass_dataset
LOCATION_NAME="$RHESSysNAME"
LOCATION="$GISDBASE"/$LOCATION_NAME
MAPSET=PERMANENT

## Modules for GIS process

### 1.1 Initiate project directory and set up GRASS GIS dataset for RHESSys modeling  

<div class="alert alert-block alert-info">
Create project directory structures and RHESSys model directory structures. We only do this once initial although these commands do not overwrite existing directories.
</div>

In [ ]:
### ... create project folder structures
mkdir "$PROJDIR"
mkdir "$PROJDIR"/grass_dataset
mkdir "$PROJDIR"/raw_data
### ... create rhessys folder structures
mkdir "$PROJDIR"/"$RHESSysNAME"
mkdir "$PROJDIR"/"$RHESSysNAME"/defs
mkdir "$PROJDIR"/"$RHESSysNAME"/flows
mkdir "$PROJDIR"/"$RHESSysNAME"/worldfiles
mkdir "$PROJDIR"/"$RHESSysNAME"/clim
mkdir "$PROJDIR"/"$RHESSysNAME"/tecfiles
mkdir "$PROJDIR"/"$RHESSysNAME"/output

### 1.2 Initiate GRASS dataset

<div class="alert alert-block alert-info">
Create project GIS database. We only do this once initial although this command does not overwrite existing database.
</div>

In [1]:
### ... create grass database
grass74 -c $EPSGCODE -e "$LOCATION" 
# it may spit out ERROR for location not available so that it can create new GIS dataset (just ignore this error).


ERROR: Error in parse(text = x, srcfile = src): <text>:2:25: unexpected string constant
1: ### ... create grass database
2: grass74 -c $EPSGCODE -e "$LOCATION"
                           ^


### 1.3 **Import and process of elevation data**

<div class="alert alert-block alert-info">
(Option 1) Assume that we need to change the elevation raster projection and resample into desired spatial resolution.<br> a) We ask GRASS to read in the elevation raster source and then resample the spatial scale<br> b) We export the sampled elevation information into a new raster "demXXm.tif", where XX is the spatial resoluted defined above<br> c) We import the "demXXm.tif" to the project GIS database with reprojection process (based on gdal) and set the project GIS database spatial extension and resolution to match the "demXXm.tif".
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.in.gdal -e --overwrite input="$PROJDIR"/raw_data/DEM1m.tif output=demRAW location=elevationRAW
LOCATIONDEM="$GISDBASE"/elevationRAW
grass74 "$LOCATIONDEM"/$MAPSET --exec g.region raster=demRAW
grass74 "$LOCATIONDEM"/$MAPSET --exec g.region res=$RESOLUTION -a -p
grass74 "$LOCATIONDEM"/$MAPSET --exec r.resamp.stats -w input=demRAW output=dem$RESOLUTION'm' ### skip this if no resampling spatial scale
grass74 "$LOCATIONDEM"/$MAPSET --exec r.out.gdal --overwrite input=dem$RESOLUTION'm' output="$PROJDIR"/raw_data/dem$RESOLUTION'm.tif' format=GTiff
grass74 "$LOCATION"/$MAPSET --exec r.in.gdal -o -e --overwrite input="$PROJDIR"/raw_data/dem$RESOLUTION'm.tif' output=dem
grass74 "$LOCATION"/$MAPSET --exec g.region raster=dem
rm -rf "$LOCATIONDEM"

<div class="alert alert-block alert-info">
(Option 2) If the source elevation raster has the correct projection (UTM) and resolution, then we can just simply import it to the project GIS dataset and set the project GIS database spatial extension and resolution to match the elevation raster.
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.in.gdal -o -e --overwrite input="$PROJDIR"/raw_data/dem$RESOLUTION'm.tif' output=dem
grass74 "$LOCATION"/$MAPSET --exec g.region raster=dem

### 1.4 **Define catchment outlet**

<div class="alert alert-block alert-info">
(Option 1) Use Lat/Long (WSG84 decimal degree) to define catchment outlet. Note that keep the negative sign if applied.
</div>

In [ ]:
gageLat='39.47947' # catchment outlet WSG84 Lat (decimal degree)
gageLong='-76.67803' # catchment outlet WSG84 Long (decimal degree; includes the negative sign if applied)
eval $(grass74 "$LOCATION"/$MAPSET --exec m.proj -i coordinates=$gageLong,$gageLat separator=space | awk '{print "xyCoord=" $1 "," $2}')
echo $xyCoord | grass74 "$LOCATION"/$MAPSET --exec v.in.ascii in=- out=outlet x=1 y=2 separator=, --overwrite

<div class="alert alert-block alert-info">
(Option 2) Import a catchment outlet shapefile. Note that this shapefile should contain only ONE point.
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec v.in.ogr --overwrite input="$PROJDIR"/raw_data/USGS_Delight.shp output=outlet location=outletRAW
LOCATIONOUTLET="$GISDBASE"'/outletRAW'
grass74 "$LOCATION"/$MAPSET --exec v.proj --overwrite location=outletRAW mapset=PERMANENT input=outlet output=outlet
rm -rf "$LOCATIONOUTLET"

### 1.5 **Catchment delineation**

<div class="alert alert-block alert-info">
User needs to provide an estimation of catchment area (m^2) and threshold values to define modeling stream network. <br>
The threshold value is an area (m^2), e.g., 620000 m^2 = 62 ha, that defines the extension of a major stream channels for RHESSys; it also affects the hillslope and subcatchment structures. <br>
The second threshold value is also an area, e.g., 100000 m^2 = 10 ha, that defines the full extension of a stream network, including head water channels, for riparian delineation and surface water routing.
</div>

In [ ]:
expectedDrainageArea=10956000 # meter sq.
expectedThresholdModelStr=620000 # meter sq.
expectedThresholdStrExt=100000 # meter sq.

In [ ]:
GRASS_thres=$(($expectedThresholdModelStr/$RESOLUTION/$RESOLUTION)) # grid cell for stream network and hillslope configuration
GRASS_thresII=$(($expectedThresholdStrExt/$RESOLUTION/$RESOLUTION))
GRASS_drainarea_lowerbound=$((98*$expectedDrainageArea/$RESOLUTION/$RESOLUTION/100)) # (allow 2% error)
GRASS_drainarea_upperbound=$((102*$expectedDrainageArea/$RESOLUTION/$RESOLUTION/100)) # (allow 2% error)
#
curl -s "$GITHUBLIBRARIES"/grass_delineation_1.sh | grass74 "$LOCATION"/$MAPSET --exec bash -s $GRASS_thres $GRASS_drainarea_lowerbound $GRASS_drainarea_upperbound
curl -s "$GITHUBLIBRARIES"/basin_determine.R | grass74 "$LOCATION"/$MAPSET --exec R --slave
grass74 "$LOCATION"/$MAPSET --exec r.watershed elevation=dem threshold=$GRASS_thresII stream=strExt --overwrite # full stream extension;
curl -s "$GITHUBLIBRARIES"/grass_delineation_2.sh | grass74 "$LOCATION"/$MAPSET --exec bash -s

<div class="alert alert-block alert-info">
When this procedure is completed, a catchment is delineated, along with slope, aspect, wettness index (D-inf), east & west horizon sun angles, hillslope IDs, subcatchment IDs, patch IDs, streams, flow accumulation, and drainage direction (D-Inf). GIS computation region and mask are also set by the delineated catchment. 
</div>

### 1.6 **Define climate zones**

<div class="alert alert-block alert-info">
(Option 1) The coarse climate zone setting  
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="zone_hill = hill"

<div class="alert alert-block alert-info">
(Option 2) Using statistical cluster analysis, we delineate the elevation bands and further partition the bands by aspect, slope, and hillslopes. The resulting climate zone is finer than the coarse setting (option 1) but not as fine as the finest setting (option 3).
</div>

In [ ]:
curl -s "$GITHUBLIBRARIES"/zone_cluster.R | grass74 "$LOCATION"/$MAPSET --exec R --slave --args dem slope aspect hill zone_cluster

<div class="alert alert-block alert-info">
(Option 3) The finest climate zone setting, same as elevation resolution. This also increases the RHESSys computation time
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="zone_patch = patch"

### 1.7 **Define isohyet (MUST)**

<div class="alert alert-block alert-info">
(Option 1) import ioshyet raster
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.in.gdal -o --overwrite input="$PROJDIR"/raw_data/isohyet.tif output=isohyet

<div class="alert alert-block alert-info">
(Option 2) calculate isohyet by raster calculator (the calculation below is just an example)
</div>

In [ ]:
grass74 $LOCATION/$MAPSET --exec r.mapcalc --overwrite expression="isohyet = (1 +(4.268e-04)*(dem-1200) +(-6.799e-05)*slope +(5.845e-03)*sin(aspect*0.01745329)*sin(slope*0.01745329) +(-1.267e-02)*cos(aspect*0.01745329)*sin(slope*0.01745329))*1.45817781"

<div class="alert alert-block alert-info">
(Option 3) no ioshyet --> set ioshyet to one everywhere
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="isohyet = 1"

### 1.8 **Soil texture from SSURGO**

<div class="alert alert-block alert-info">
This step requires the SSURGO spatial and tabular dataset. <br>
Users need to set the path to the SSURGO dataset, e.g., "$PROJDIR"/raw_data/MD005 <br>
The procedure below reprojects the SSURGO mukey (areal polygon) shapefile onto the project GIS database, extract soil information from the tabular dataset, and map the extracted soil information to the corresponding mukey polygons.
</div>

<div class="alert alert-block alert-info">
(Option 1) Simple soil texture classification; see USDA soil classes for details
</div>

In [ ]:
downloadedSSURGOdirectoryPATH="$PROJDIR"/raw_data/MD005
grass74 "$LOCATION"/$MAPSET --exec v.in.ogr --overwrite input="$downloadedSSURGOdirectoryPATH"/spatial/soilmu_a_"$(echo ${downloadedSSURGOdirectoryPATH##*/} | tr '[A-Z]' '[a-z]')".shp output=ssurgo location=soilRAW
LOCATIONSOIL="$GISDBASE"/soilRAW
### ... import (reprojected) soil data to ""$LOCATION"/$MAPSET" and convert shape-polygon to raster soil (USDA) classes
grass74 "$LOCATION"/$MAPSET --exec v.proj --overwrite location=soilRAW mapset=PERMANENT input=ssurgo output=ssurgo
grass74 "$LOCATION"/$MAPSET --exec v.to.rast --overwrite input=ssurgo use=cat output=soil_ssurgo
grass74 "$LOCATION"/$MAPSET --exec v.out.ascii --overwrite input=ssurgo type=centroid output="$PROJDIR"/"$RHESSysNAME"/soil_cat_mukey.csv columns=MUKEY format=point separator=comma

In [ ]:
curl -s https://raw.githubusercontent.com/laurencelin/ssurgo_extraction/master/ssurgo_extraction.R | R --slave --args "$downloadedSSURGOdirectoryPATH"
curl -s https://raw.githubusercontent.com/laurencelin/ssurgo_extraction/master/ssurgo_soiltexture2gis.R | grass74 "$LOCATION"/$MAPSET --exec R --slave --args "$PROJDIR"/"$RHESSysNAME"/soil_cat_mukey.csv "$downloadedSSURGOdirectoryPATH"/soil_mukey_texture.csv
rm -rf "$LOCATIONSOIL"

<div class="alert alert-block alert-info">
If filling missing data or gaps is needed, below is a command for the job. Please read documentations of "r.fill.stats", a build-in GRASS function, to know how to use it correct.
</div>

In [ ]:
grass74 $LOCATION/$MAPSET --exec r.fill.stats -k --overwrite input=soil_texture output=soil_texture_fill distance=30 mode=mode power=2.0

<div class="alert alert-block alert-info">
(Option 2) Explicit soil information for local catchment, exacted from SSURGO database
</div>

In [ ]:
downloadedSSURGOdirectoryPATH="$PROJDIR"/raw_data/MD005
grass74 "$LOCATION"/$MAPSET --exec v.in.ogr --overwrite input="$downloadedSSURGOdirectoryPATH"/spatial/soilmu_a_"$(echo ${downloadedSSURGOdirectoryPATH##*/} | tr '[A-Z]' '[a-z]')".shp output=ssurgo location=soilRAW
LOCATIONSOIL="$GISDBASE"/soilRAW
### ... import (reprojected) soil data to ""$LOCATION"/$MAPSET" and convert shape-polygon to raster soil (USDA) classes
grass74 "$LOCATION"/$MAPSET --exec v.proj --overwrite location=soilRAW mapset=PERMANENT input=ssurgo output=ssurgo
grass74 "$LOCATION"/$MAPSET --exec v.to.rast --overwrite input=ssurgo use=cat output=soil_ssurgo
grass74 "$LOCATION"/$MAPSET --exec v.out.ascii --overwrite input=ssurgo type=centroid output="$PROJDIR"/"$RHESSysNAME"/soil_cat_mukey.csv columns=MUKEY format=point separator=comma

In [ ]:
curl -s https://raw.githubusercontent.com/laurencelin/ssurgo_extraction/master/ssurgo_fullextraction_step1.R | R --slave --args "$downloadedSSURGOdirectoryPATH"
curl -s https://raw.githubusercontent.com/laurencelin/ssurgo_extraction/master/ssurgo_fullextraction_step2.R | R --slave --args "$downloadedSSURGOdirectoryPATH" "$PROJDIR"/"$RHESSysNAME"/soil_cat_mukey.csv "$PROJDIR"/"$RHESSysNAME"/soil_mukey_rhessys.csv
curl -s https://raw.githubusercontent.com/laurencelin/ssurgo_extraction/master/ssurgo_fullextraction_step3.R | grass74 "$LOCATION"/$MAPSET --exec R --slave --args soil_ssurgo "$PROJDIR"/"$RHESSysNAME"/soil_mukey_rhessys.csv
rm -rf "$LOCATIONSOIL"

<div class="alert alert-block alert-info">
If filling missing data or gaps is needed, below is a command for the job. Please read documentations of "r.fill.stats", a build-in GRASS function, to know how to use it correct.
</div>

### 1.9 **Processing Landuse & landcover data**

<div class="alert alert-block alert-info">
An important concept: Landuse & landcover (LULC) data provide % coverages of pre-defined classes on each grid (30, 10, 5, 3, 1 meters). These pre-defined classes could be very general. For stances, deciduous forest class (code 41 in NLCD) does not specifically define which decidous species or their composition to form that forest, nor provide the forest density / LAI value. All we learned from the 30-m coded 41 grid is that it's largely covered by deciduous forest. This is the reason why we need to specifically define the forest for the model in the next step. Another example, urban class 23 provides two pieces of information: urbanized landscape and median density of urbanization. These information about urban are imprecise for the model. To describe urbanization to the model, we need to let the model know how much rooftop covered area, how much imprevious surfaces, how much paved road covered, how storm drainage connect to different parts of landscape, and how much urban canopy still remained within the urbanized area. This motivates us to use high-resolution LULC dataset (e.g., EPA EnviroAtlas and Chesapeake Conservancy) with great detailed and specific LULC classes. To make use of these great detailed LULC information at the modeling scale (3, 10, or 30 m), we need to aggregate and lump information into the following compositions for each modeling patch:<br>
- forest composition (forestFrac) <br>
- shrub composition (shurbFrac) <br>
- crop composition (cropFrac) <br>
- grass composition (lawnFrac) <br>
- imperviousness composition (impFrac) <br>
<br>
For imperviousness compositional fraction, we further partition this fraction into different catagories:<br>
- impervious structure roof (roofFrac) <br>
- impervious parking/non-structural surface (drivewayFrac) <br>
- impervious (paved) road (pavedroadFrac) <br>
<br>
</div>

<div class="alert alert-block alert-info">
To translate LULC classifications into the compositions above, we developed an approach to define each pre-defined LULC class in the dataset by the compositions. We illustrated that with an example below.
</div>

For example, NLCD code 42 - deciduous forest, 22 low-density urban, and 24 high-density urban,

| classification code | forestComp | shrubComp | CropComp | grassComp | impComp |
|------|------|------|------|------|
|   42 | 1| 0| 0| 0| 0|
|   22 | 0.2| 0| 0| 0.3|0.5 |
|   24 | 0| 0| 0| 0| 1|

<div class="alert alert-block alert-info">
Above is the concept, and below lists different modules to accommodate different LULC sources
</div>

<div class="alert alert-block alert-info">
(Option 1) NLCD LULC classification for remotely forested landscape
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.in.gdal -e --overwrite input="$PROJDIR"/raw_data/'LULC1m.tif' output=lulcRAW location=lulcRAW
LOCATIONLULC="$GISDBASE"/lulcRAW
grass74 "$LOCATION"/$MAPSET --exec r.to.vect input=patch output=patch type=area
grass74 "$LOCATIONLULC"/$MAPSET --exec v.proj location=$LOCATION_NAME mapset=$MAPSET input=patch output=patch$RESOLUTION'm'
grass74 "$LOCATIONLULC"/$MAPSET --exec v.to.rast input=patch$RESOLUTION'm' output=patch$RESOLUTION'm' use=attr attribute_column=value
#
grass74 "$LOCATIONLULC"/$MAPSET --exec g.region zoom=patch$RESOLUTION'm'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac.R | grass74 "$LOCATIONLULC"/$MAPSET --exec R --slave --args patch$RESOLUTION'm' lulcRAW "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac_write2GIS.R | grass74 "$LOCATION"/$MAPSET --exec R --slave --args patch "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' 'https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/lulc_30m_NLCD_remote_forest_catchment.csv'
#
rm -rf "$LOCATIONLULC"

<div class="alert alert-block alert-info">
(Option 2) NLCD LULC classification for urbanized landscape
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.in.gdal -e --overwrite input="$PROJDIR"/raw_data/'LULC1m.tif' output=lulcRAW location=lulcRAW
LOCATIONLULC="$GISDBASE"/lulcRAW
grass74 "$LOCATION"/$MAPSET --exec r.to.vect input=patch output=patch type=area
grass74 "$LOCATIONLULC"/$MAPSET --exec v.proj location=$LOCATION_NAME mapset=$MAPSET input=patch output=patch$RESOLUTION'm'
grass74 "$LOCATIONLULC"/$MAPSET --exec v.to.rast input=patch$RESOLUTION'm' output=patch$RESOLUTION'm' use=attr attribute_column=value
#
grass74 "$LOCATIONLULC"/$MAPSET --exec g.region zoom=patch$RESOLUTION'm'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac.R | grass74 "$LOCATIONLULC"/$MAPSET --exec R --slave --args patch$RESOLUTION'm' lulcRAW "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac_write2GIS.R | grass74 "$LOCATION"/$MAPSET --exec R --slave --args patch "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' 'https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/lulc_30m_NLCD_urban_catchment.csv'
#
rm -rf "$LOCATIONLULC"

<div class="alert alert-block alert-info">
(Option 3) Baltimore LULC classification
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.in.gdal -e --overwrite input="$PROJDIR"/raw_data/'LULC1m.tif' output=lulcRAW location=lulcRAW
LOCATIONLULC="$GISDBASE"/lulcRAW
grass74 "$LOCATION"/$MAPSET --exec r.to.vect input=patch output=patch type=area
grass74 "$LOCATIONLULC"/$MAPSET --exec v.proj location=$LOCATION_NAME mapset=$MAPSET input=patch output=patch$RESOLUTION'm'
grass74 "$LOCATIONLULC"/$MAPSET --exec v.to.rast input=patch$RESOLUTION'm' output=patch$RESOLUTION'm' use=attr attribute_column=value
#
grass74 "$LOCATIONLULC"/$MAPSET --exec g.region zoom=patch$RESOLUTION'm'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac.R | grass74 "$LOCATIONLULC"/$MAPSET --exec R --slave --args patch$RESOLUTION'm' lulcRAW "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac_write2GIS.R | grass74 "$LOCATION"/$MAPSET --exec R --slave --args patch "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' 'https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/lulc_1m_Chesapeake_Conservancy.csv'
#
rm -rf "$LOCATIONLULC"

<div class="alert alert-block alert-info">
(Option 4) Virginia LULC classification
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.in.gdal -e --overwrite input="$PROJDIR"/raw_data/'LULC1m.tif' output=lulcRAW location=lulcRAW
LOCATIONLULC="$GISDBASE"/lulcRAW
grass74 "$LOCATION"/$MAPSET --exec r.to.vect input=patch output=patch type=area
grass74 "$LOCATIONLULC"/$MAPSET --exec v.proj location=$LOCATION_NAME mapset=$MAPSET input=patch output=patch$RESOLUTION'm'
grass74 "$LOCATIONLULC"/$MAPSET --exec v.to.rast input=patch$RESOLUTION'm' output=patch$RESOLUTION'm' use=attr attribute_column=value
#
grass74 "$LOCATIONLULC"/$MAPSET --exec g.region zoom=patch$RESOLUTION'm'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac.R | grass74 "$LOCATIONLULC"/$MAPSET --exec R --slave --args patch$RESOLUTION'm' lulcRAW "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac_write2GIS.R | grass74 "$LOCATION"/$MAPSET --exec R --slave --args patch "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' 'https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/lulc_1m_va.csv'
#
rm -rf "$LOCATIONLULC"

<div class="alert alert-block alert-info">
(Option 5) For other customized LULC data, 
this block of code below is to create an csv table for users to fill in the composition information for each LULC class
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.in.gdal -e --overwrite input="$PROJDIR"/raw_data/'LULC1m.tif' output=lulcRAW location=lulcRAW
LOCATIONLULC="$GISDBASE"/lulcRAW
grass74 "$LOCATION"/$MAPSET --exec r.to.vect input=patch output=patch type=area
grass74 "$LOCATIONLULC"/$MAPSET --exec v.proj location=$LOCATION_NAME mapset=$MAPSET input=patch output=patch$RESOLUTION'm'
grass74 "$LOCATIONLULC"/$MAPSET --exec v.to.rast input=patch$RESOLUTION'm' output=patch$RESOLUTION'm' use=attr attribute_column=value
#
grass74 "$LOCATIONLULC"/$MAPSET --exec g.region zoom=patch$RESOLUTION'm'
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac.R | grass74 "$LOCATIONLULC"/$MAPSET --exec R --slave --args patch$RESOLUTION'm' lulcRAW "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv'
curl -s "$GITHUBLIBRARIES"/LULC_codeinformation.R | R --slave --args "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' "$PROJDIR"/"$RHESSysNAME"/lulc_codeinformation.csv
rm -rf "$LOCATIONLULC"

<div class="alert alert-block alert-info">
Fill the csv table $PROJDIR / $RHESSysNAME/lulc_codeinformation.csv
</div>

In [ ]:
curl -s "$GITHUBLIBRARIES"/aggregate_lulcFrac_write2GIS.R | grass74 "$LOCATION"/$MAPSET --exec R --slave --args patch "$PROJDIR"/"$RHESSysNAME"/lulcFrac$RESOLUTION'm.csv' "$PROJDIR"/"$RHESSysNAME"/lulc_codeinformation.csv

### 1.10 **Define forest vegetation**

<div class="alert alert-block alert-info">
The forest/shrub/crop/grass LULC composition calculated above is a defined area for forest/shrub/crop/grass, and we have to define what vegetations should be growing in the area. Current script can support up to 15 vegetation types per patch.
</div>

<div class="alert alert-block alert-info">
(Option 1)
For example, we model an oak-dominated canopy in a forested catchment. <br>
We determine the locations of oak canopy (the first line) <br>
We simply model the full canopy as oak and maple (the second line); "FFrac" indicates the oak composition in the full canopy. In this example, FFrac = 0.8, i.e., we treat the full canopy as 80% oak and 20% maple.<br>
We set the LAI for the oak canopy (the third line). In this example, LAI is a constant 5.5. One can substitude the 5.5 value by an imported remotely sensed LAI raster. 
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="oak_canopy = if(forestFrac>0,102,null())" # - the first line
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="oak_canopy_FFrac = if(forestFrac>0,0.8,null())" # - the second line
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="oak_canopy_LAI = if(forestFrac>0,5.5,null())" # - the third line

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="maple_canopy = if(forestFrac>0,111,null())"
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="maple_canopy_FFrac = if(forestFrac>0,0.2,null())"
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="maple_canopy_LAI = if(forestFrac>0,5.5,null())"

<div class="alert alert-block alert-info">
Similarly, we can model the lawn/pasture the same way.
</div>

In [ ]:
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="grass1StratumID = if(lawnFrac>0,3,null())"
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="grass1FFrac = if(lawnFrac>0,1.0,null())"
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="grass1LAI = if(lawnFrac>0,1.5,null())"

<div class="alert alert-block alert-info">
(Option 2) (update later ... )
</div>

### 1.11 **Import road network (shapefile; if applied)**

<div class="alert alert-block alert-info">
This is optional to import a vector-based road network.
</div>

In [ ]:
downloadedROADfile="$PROJDIR"/raw_data/'Roads_GDT_MSA.shp'
grass74 "$LOCATION"/$MAPSET --exec v.in.ogr --overwrite input="$downloadedROADfile" output=roads location=roadRAW
LOCATIONROAD="$GISDBASE"/roadRAW
grass74 "$LOCATION"/$MAPSET --exec v.proj --overwrite location=roadRAW mapset=$MAPSET input=roads output=roads
grass74 "$LOCATION"/$MAPSET --exec v.to.rast --overwrite input=roads output=vector_roads use=cat
rm -rf "$LOCATIONROAD"

### 1.12 **Define riparian (if applied)**

<div class="alert alert-block alert-info">
(Option 1) import a determined riparian raster
</div>

In [ ]:
grass74 "$LOCATION"/"$MAPSET" --exec r.in.gdal -o --overwrite input="$PROJDIR"/raw_data/riparian.tif output=riparian

<div class="alert alert-block alert-info">
(Option 2) Use HANDS tool to define riparian zone. This is timely process.
</div>

In [ ]:
curl -s "$GITHUBLIBRARIES"/elevation_analysis.R | grass74 "$LOCATION"/$MAPSET --exec R --slave --args dem colmap rowmap drain hill strExt
grass74 "$LOCATION"/"$MAPSET" --exec r.mapcalc expression="riparian_hands = if( handsDEM < 5, 1, null())" --overwrite

### 1.13 **Define surface drainage features for urban catchments (if applied)**

<div class="alert alert-block alert-info">
Current RHESSys EC supports storm drinage along road network, (subsurface) sewer drinage, and surface drainage. 
</div>

In [ ]:
### ... storm drinage along the paved roads
grass74 "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roadDEM = if(pavedroadFrac>0,dem,null())"
grass74 "$LOCATION"/"$MAPSET" --exec r.watershed -s --overwrite elevation=roadDEM drainage=roadDrain
grass74 "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roadExit = if(roadDrain<0, patch, null())"
grass74 "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roadExitDEM = if(isnull(roadExit),null(),dem)"
grass74 "$LOCATION"/"$MAPSET" --exec r.to.vect --overwrite input=roadExit output=roadExit type=point column=inPatch
#
grass74 "$LOCATION"/"$MAPSET" --exec r.buffer --overwrite input=roadExit output=roadExit_buffer distances=120
grass74 "$LOCATION"/"$MAPSET" --exec r.buffer --overwrite input=roadDEM output=road_buffer distances=30
grass74 "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roadExit_neighbor_area = if(roadExit_buffer>1 & isnull(road_buffer),  if(isnull(strExt) & impFrac==0,1, null()), null())"
grass74 "$LOCATION"/"$MAPSET" --exec r.clump --overwrite -d input=roadExit_neighbor_area output=roadExit_neighbor_id
grass74 "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roadExit_neighbor_usdDEM = if(roadExit_buffer>1 & isnull(road_buffer),  if(isnull(strExt) & impFrac==0,usdDEM, null()), null())"
curl -s "$GITHUBLIBRARIES"/roadstormdriangeOutlet.R | grass74 "$LOCATION"/$MAPSET --exec R --slave --args basin roadExitDEM roadExit_neighbor_id roadExit_neighbor_usdDEM dem xmap ymap patch roadExit_neighbor_usdDEMmed roadExitOutletPatchID roadExitOutlet 120
grass74 "$LOCATION"/"$MAPSET" --exec r.to.vect --overwrite input=roadExitOutlet output=roadExitOutlet type=point column=inPatch

In [ ]:
### ... rooftop surface routing
grass74 "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roofQ = if(roofFrac>0,1,null())"
grass74 "$LOCATION"/"$MAPSET" --exec r.grow --overwrite input=roofQ output=roofQgrow radius=1.51
grass74 "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roofDEM = if(isnull(roofQgrow),null(),if(isnull(roofQ),dem,dem+6))"
grass74 "$LOCATION"/"$MAPSET" --exec r.watershed -s --overwrite elevation=roofDEM drainage=roofDrain_
grass74 "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roofDrain = if(isnull(roofQ),null(),roofDrain_)"
### ... driveway surface routing
grass74 "$LOCATION"/"$MAPSET" --exec r.mapcalc --overwrite expression="roaddrivewayDEM = if(pavedroadFrac_va1mFilled>0,if(drivewayFrac_va1mFilled>0, dem, null()),null())"
grass74 "$LOCATION"/"$MAPSET" --exec r.watershed -s --overwrite elevation=roaddrivewayDEM drainage=roaddrivewayDrain

In [ ]:
### ... assume sewer lines go along major roads (imported from shapefile) and sewer lines drainge a neigbour area
grass74 "$LOCATION"/$MAPSET --exec r.buffer --overwrite input=roads output=roadbuff distances=30
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc expression="sewercover = if( (lawnFrac>0.1 || impFrac >0.1) && pavedRoadFrac<0.3 && roadbuff>0, 1, null())" --overwrite

In [ ]:
### ... additional rapid surface drainage  (other than roof top, driveway, parking, and paved roads), e.g., drainage hole in playground / garden
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="addsurfdrain = if( forestFrac<0.5 && roofBuff>0, 1 , null())"

In [ ]:
### ... soil compactness by construction
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="roof = if(roofFrac > 0.5, 1, null())"
grass74 "$LOCATION"/$MAPSET --exec r.buffer --overwrite input=roof output=roofBuff distances=30
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="naturalLand = if( (forestFrac>=1 || lawnFrac>=1)&& isnull(roofBuff), 1 , null())"
grass74 "$LOCATION"/$MAPSET --exec r.mapcalc --overwrite expression="compactedsoil = if( naturalLand>0, null(), basin)"

## Part 2: template of listing maps/layers to build RHESSysEC model

### 2.1 **Template keywords**

<div class="alert alert-block alert-info">
Below lists keywords in the template. Each keyword should follow by a space and the name of a map in GIS
</div>

In [ ]:
# ... paths to ouputs and parameter libraries
outputWorldfile [path to the output] 1 
outputWorldfileHDR [path to the output] 1
outputDefs [path to the output] 1
outputSurfFlow [path to the output] 1
outputSubFlow [path to the output] 1
vegCollection [path to the parameter library]
soilCollection [path to the parameter library]
lulcCollection [path to the parameter library]
#
# ... spatial hierarchical structure (must-have)
basinMap [name of a map in GIS]
hillslopeMap 
zoneMAP
patchMAP
# ... soil characters (must-have)
soilidMAP [name of a map in GIS]
soiltexture
soilksat0
soilksatdecay
soilpor0
soilpordecay
soilsand
soilsilt
soilclay
soilbulkdensity
soilparticledensity
soilsoildepth
soilactivedepth
soilmaxrootdepth
soilalbedo
soilpor_size_index
soilpsi_air_entry
soilsoilc
soilomdecay
# ... topographic variables (must-have)
xMAP [name of a map in GIS]
yMAP
demMAP
slopeMap
aspectMAP
twiMAP
whorizonMAP
ehorizonMAP
isohyetMAP
rowMap
colMap
drainMap
# ... patch total impervious (prevent infiltration & returnflow; change detention storage capacity)
impFracMAP [name of a map in GIS] # (must-have)
# ... forest vegetations [1...N]
forestFracMAP [name of a map in GIS] # (must-have)
tree1StratumID
tree1FFrac 
tree1LAI
tree2StratumID
tree2FFrac 
tree2LAI
# ... shrub vegetation [1...N]
shrubFracMAP # (must-have)
shrub1StratumID
shrub1FFrac
shrub1LAI
shrub2StratumID
shrub2FFrac
shrub2LAI
# ... crop vegetation [1...N]
cropFracMAP # (must-have)
crop1StratumID
crop1FFrac
crop1LAI
crop2StratumID
crop2FFrac
crop2LAI
# ... lawn/pasture vegetation [1...N]
grassFracMAP # (must-have)
grass1StratumID
grass1FFrac
grass1LAI
grass2StratumID
grass2FFrac
grass2LAI
# ------------ The following maps are routing related ------------- #
#
# ... surface routing -- stream 
streamMap str  # <-  (must-have) modeling stream-grids
streamFullExtension  # <- non-modeled surface drain in the channel (i.e., very small streams)
#
# ... surface routing -- road/driveway storm drainage
otherImpMAP  # <- e.g., driveway, parking space, private roads, sidewalks, ... etc
pavedRoadFracMAP # <- major paved road network
roadStormDrainInlet # <- inlet patchID at the inlet location; !isnull() triggers actionSTORMDRAIN;
roadStormDrainOutlet # <- outlet patchID at the inlet location
onRoadDraingeDir # <- drain directions
#
## ... surface routing -- surface drainage via pipes
additionalSurfaceDrainInletMAP # <- inlet patchID at the inlet location
additionalSurfaceDrainOutlet # <- outlet patchID at the inlet location
#
## ... surface routing -- housing
roofMAP  # <- prevent surrounding surface in-coming water to the rooftops
roofDrainMAP # <- drain directions
roofRedirectedInlet # <- inlet patchID at the inlet location
roofRedirectedOutlet # <- outlet patchID at the inlet location; may be a water tank / raingarden / drainage design; but one outlet per roof grid
#
## ... subsurface drainages -- pipelines and sewers
sewercoverMAP  # <- !isnull() triggers actionSEWER; take out from streamflow
pipecoverMAP  # <- !isnull() triggers actionPIPEDRAIN; add back to streamflow but regulating watertable
#
## ... subsurface -- interrcepts
compactedSoilMAP 
unpavedRoadMap 
basementFracMAP

## ... surface/subsurface -- transfer
riparianMAP  # <- !isnull() triggers actionRIPARIAN;
septicMAP # <- (developing) lumped non-zero value at outlet location; use lulc def parameter if no source
septicSourceMAP # <- (developing) lumped non-zero value at source location;
irrigrationSourceMAP # <- (developing) non-null / non-zero value at source location;
irrigrationMAP  # <- (developing) non-null / non-zero value at outlet location; use lulc def parameter if no source; bounded by crop and lawn Frac



<div class="alert alert-block alert-info">
Below is an example to generate a template by commandline. Certainly, one can just manually construct a template file using the keywords and format described above. Assume that we have the header in section 0 defined in the commandline environment
</div>

In [ ]:
templateFile="$PROJDIR"/"$RHESSysNAME"/g2w_template.txt
##
# set paths for RHESSys input files
# 1 = Yes, output this file; 0 = No, do not output this file
echo outputWorldfile \""$PROJDIR"/"$RHESSysNAME"/worldfiles/worldfile.csv\" 1 > "$templateFile"
echo outputWorldfileHDR \""$PROJDIR"/"$RHESSysNAME"/worldfiles/worldfile.hdr\" 1 >> "$templateFile"
echo outputDefs \""$PROJDIR"/"$RHESSysNAME"/defs\" 1 >> "$templateFile"
echo outputSurfFlow \""$PROJDIR"/"$RHESSysNAME"/flows/surfflow.txt\" 1 >> "$templateFile"
echo outputSubFlow \""$PROJDIR"/"$RHESSysNAME"/flows/subflow.txt\" 1 >> "$templateFile"
echo vegCollection \"https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/vegCollection.csv\" >> "$templateFile"
echo soilCollection \"https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/soilCollection.csv\" >> "$templateFile"
echo lulcCollection \"https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/lulcCollectionEC.csv\" >> "$templateFile"
#
# set climate station ID and file name
echo stationID 101 >> "$templateFile"
echo stationFile \""clim/stan.base"\" >> "$templateFile"
#
# the following maps that must be provided with syntex:
# echo keyword <map> >> "$templateFile"
echo basinMap basin >> "$templateFile"
echo hillslopeMap hill >> "$templateFile"
echo zoneMAP zone_cluster >> "$templateFile"
echo patchMAP patch >> "$templateFile"
#
echo soilidMAP soil_ssurgoFILL >> "$templateFile"
echo soiltexture soil_textureFILL >> "$templateFile"
echo soilksat0 soil_ksat_0FILL >> "$templateFile"
echo soilksatdecay soil_ksat_decayFILL >> "$templateFile"
echo soilpor0 soil_por_0FILL >> "$templateFile"
echo soilpordecay soil_por_decayFILL >> "$templateFile"
echo soilsand soil_sandFILL >> "$templateFile"
echo soilsilt soil_siltFILL >> "$templateFile"
echo soilclay soil_clayFILL >> "$templateFile"
echo soilbulkdensity soil_bulkdensityFILL >> "$templateFile"
echo soilparticledensity soil_particledensityFILL >> "$templateFile"
echo soilsoildepth soil_soildepthFILL >> "$templateFile"
echo soilactivedepth soil_activedepthFILL >> "$templateFile"
echo soilmaxrootdepth soil_maxrootdepthFILL >> "$templateFile"
echo soilalbedo soil_albedoFILL >> "$templateFile"
echo soilpor_size_index soil_por_size_indexFILL >> "$templateFile"
echo soilpsi_air_entry soil_psi_air_entryFILL >> "$templateFile"
echo soilsoilc soil_soilcFILL >> "$templateFile"
echo soilomdecay soil_omdecayFILL >> "$templateFile"
#
echo xMAP xmap >> "$templateFile"
echo yMAP ymap >> "$templateFile"
echo demMAP dem >> "$templateFile"
echo slopeMap slope >> "$templateFile"
echo aspectMAP aspect >> "$templateFile"
echo twiMAP wetness_index >> "$templateFile"
echo whorizonMAP west_180 >> "$templateFile"
echo ehorizonMAP east_000 >> "$templateFile"
echo isohyetMAP isohyet >> "$templateFile"
echo rowMap rowmap >> "$templateFile"
echo colMap colmap >> "$templateFile"
echo drainMap drain >> "$templateFile"
# ... patch total impervious
echo impFracMAP impFrac >> "$templateFile"
# ... forest vegetations
echo forestFracMAP forestFrac_community >> "$templateFile"
#
echo tree1StratumID pine_vegid >> "$templateFile"
echo tree1FFrac pine_Ffrac >> "$templateFile"
echo tree1LAI pine_lai_calc >> "$templateFile"
#
echo tree2StratumID oak_vegid >> "$templateFile"
echo tree2FFrac oak_Ffrac >> "$templateFile"
echo tree2LAI oak_lai_calc >> "$templateFile"
#
echo tree3StratumID hemlock_vegid >> "$templateFile"
echo tree3FFrac hemlock_Ffrac >> "$templateFile"
echo tree3LAI hemlock_lai_calc >> "$templateFile"
#
echo tree4StratumID maple_vegid >> "$templateFile"
echo tree4FFrac maple_Ffrac >> "$templateFile"
echo tree4LAI maple_lai_calc >> "$templateFile"
#
echo tree5StratumID tulip_vegid >> "$templateFile"
echo tree5FFrac tulip_Ffrac >> "$templateFile"
echo tree5LAI tulip_lai_calc >> "$templateFile"
#
# ... shrub vegetation
echo shrubFracMAP shrubFrac >> "$templateFile"
echo shrub1StratumID shrub_vegid >> "$templateFile"
echo shrub1FFrac shrub_Ffrac >> "$templateFile"
echo shrub1LAI shrub_lai >> "$templateFile"
# ... crop vegetation
#echo cropFracMAP cropFrac >> "$templateFile"
#echo crop1StratumID crop1StratumID >> "$templateFile"
#echo crop1FFrac crop1FFrac >> "$templateFile"
#echo crop1LAI crop1LAI >> "$templateFile"
# ... lawn/pasture vegetation
echo grassFracMAP lawnFrac >> "$templateFile"
#echo grass1StratumID grass1StratumID >> "$templateFile"
#echo grass1FFrac grass1FFrac >> "$templateFile"
#echo grass1LAI grass1LAI >> "$templateFile"
#
# ------------ The following maps are routing related ------------- #
#
## ... surface routing -- stream 
echo streamMap str_adj >> "$templateFile" # ... modeling stream-grids
#
## ... surface routing -- road storm drainage
#echo otherImpMAP drivewayFrac >> "$templateFile" # e.g., driveway, parking space, private roads, sidewalks, ... etc
#echo pavedRoadFracMAP pavedroadFrac >> "$templateFile"
#echo roadStormDrainInlet roadExit >> "$templateFile"
#echo roadStormDrainOutlet roadExitOutletPatchID >> "$templateFile"
#echo onRoadDraingeDir roaddrivewayDrain >> "$templateFile"
#
## ... surface routing -- housing & drive way
#echo roofMAP roofFrac >> "$templateFile" # stopping surface in-coming water
#echo roofDrainMAP roofDrain >> "$templateFile" 
#
## ... surface/subsurface -- transfer
echo riparianMAP riparian_hands >> "$templateFile" ## --> gw-to-riparian
#

### 2.2 **Building RHESSysEC model with a template**

<div class="alert alert-block alert-info">
Use simple soil texture information
</div>

In [ ]:
curl -s "$GITHUBLIBRARIES"/g2w_cf_RHESSysEC_soil_texture.R | grass74 "$LOCATION"/$MAPSET --exec R --slave --args "$PROJDIR" default default default "$templateFile"
curl -s "$GITHUBLIBRARIES"/LIB_RHESSys_writeTable2World_soil_fullextraction.R | R --slave --args na "$PROJDIR"/"$RHESSysNAME"/worldfiles/worldfile.csv "$PROJDIR"/"$RHESSysNAME"/worldfiles/worldfile

<div class="alert alert-block alert-info">
Use explicit soil information
</div>

In [ ]:
curl -s "$GITHUBLIBRARIES"/g2w_cf_RHESSysEC_soil_fullextraction.R | grass74 "$LOCATION"/$MAPSET --exec R --slave --args "$templateFile"
curl -s "$GITHUBLIBRARIES"/LIB_RHESSys_writeTable2World_soil_fullextraction.R | R --slave --args na "$PROJDIR"/"$RHESSysNAME"/worldfiles/worldfile.csv "$PROJDIR"/"$RHESSysNAME"/worldfiles/worldfile